Link da documentação https://platform.openai.com/docs/api-reference/introduction

Nos testes, não foi utilizado o pacote disponibilizado pela OpenAI para Python. Em vez disso, foi utilizada uma requisição via requests, pois essa abordagem pode ser aplicada em qualquer linguagem de programação.

Lista de comandos para linux ou mac
- Criar o anbiete virtual python -m venv nome_do_ambiente 
- Iniciar o anbiente python -m venv nome_do_ambiente (Para linux ou mac)
- Instalar os pacotes utilizado no abiente pip install -r requirements.tx




In [23]:
from dotenv import load_dotenv
import os
import requests
import json
import csv
import pandas as pd
from IPython.display import display

In [24]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Acessa a variável de ambiente API_KEY
API_KEY = os.getenv('API_KEY')

# Verifique se a API_KEY foi carregada corretamente
if not API_KEY:
    raise ValueError("API_KEY não foi carregada corretamente. Verifique o arquivo .env")



In [25]:

#Verificando os modelos disponibilizados para utilização
#API_KEY  esta variável tem o codígo da api da openai
# Define os headers e a URL da API
headers = {"Authorization": f"Bearer {API_KEY}"}
url = 'https://api.openai.com/v1/models'
# Faz a requisição GET para listar todos os modelos
response = requests.get(url, headers=headers)
# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    models = response.json()

    # Especifica o nome do arquivo CSV
    csv_file = 'modelos_openai.csv'
    # Escreve os dados no arquivo CSV
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Escreve o cabeçalho
        writer.writerow(['Model ID', 'Object'])
        # Escreve os dados dos modelos
        for model in models['data']:
            writer.writerow([model['id'], model['object']])

    print(f"Lista de modelos salva no arquivo {csv_file}")
else:
    print(f"Falha ao recuperar modelos. Código de status: {response.status_code}")
    print(response.text)

Lista de modelos salva no arquivo modelos_openai.csv


In [26]:
#listando os modelos
df = pd.read_csv('modelos_openai.csv')
display(df)

,Model ID,Object
0,dall-e-3,model
1,gpt-4-1106-preview,model
2,whisper-1,model
3,gpt-4o-2024-05-13,model
4,babbage-002,model
5,dall-e-2,model
6,gpt-3.5-turbo-16k,model
7,tts-1-hd-1106,model
8,tts-1-hd,model
9,gpt-4o,model


In [ ]:
# Define os headers e a URL da API
headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
link = 'https://api.openai.com/v1/chat/completions'

 # Em model coloque o nome do modelo que deseja utilizar
 # Em messages o role coresponde ao usuário que esta enviado a mensagem o content o conteudo
 
nome_produto = "Tinta 3,6 litros"  
body_mensagem = {
    "model": "gpt-3.5-turbo-16k",   
    "messages": [{"role": "user", 
                  "content": f"O item {nome_produto}, Pertence a qual categoria de produto.? Na sua resposta coloque somente o nome da categoria."}]
}
#Ajustando para formato json para o post
body_mensagem = json.dumps(body_mensagem)

# Faz a requisição POST
response = requests.post(link, headers=headers, data=body_mensagem)

# Imprime a resposta da requisição
# A responta fica em choices - message - content
# Note que na resposta o role tem o nome de 
print(response)
print(response.text)

: 

In [48]:
#Convertendo o json para acessar a resposta
resposta = response.json()
mensagem = resposta['choices'][0]['message']['content']
print(mensagem)

Papelaria
